# K-means Clustering

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Finding closest centroids
In the “cluster assignment” phase of the K-means algorithm, the algorithm assigns every training example 𝑥(𝑖) to its closest centroid, given the current positions of centroids.

In [ ]:
def find_closest_centroids(X, centroids):
    """
    Computes the centroid memberships for every example
    
    Args:
        X (ndarray): (m, n) Input values      
        centroids (ndarray): k centroids
    
    Returns:
        idx (array_like): (m,) closest centroids
    
    """
    K = centroids.shape[0]

    idx = np.zeros(X.shape[0], dtype=int)

    a = np.linalg.norm(X[:, None] - centroids, axis=2)
    idx = np.argmin(a, axis=1)
    
    return idx